GRUPO 1 PySpark Dataset 1

Nicolasa Nancy Illanes Castillo

Gerick Toro

Juan Jose Velarde


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import Imputer, StandardScaler, VectorAssembler


# Inicializar Spark
spark = SparkSession.builder \
    .appName("DataQualityExample") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Cargar datos
df_spark = spark.read.option("header", "true").csv("data.csv", inferSchema=True)
df_spark.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [3]:
# analisis de calidad

def spark_quality_analysis(df):

    print(" ANÁLISIS CALIDAD DATOS   ")

    # Conteo de registros
    print(f"Numero de total registros: {df.count()}")
    print(f"Total columnas: {len(df.columns)}")

    # Análisis de nulls por columna
    null_counts = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c)
        for c in df.columns  ]).collect()[0]

    print("\nVALORES NULOS ")
    for col_name, null_count in null_counts.asDict().items():
        if null_count > 0:
            percentage = (null_count / df.count()) * 100
            print(f"{col_name}: {null_count} ({percentage:.2f}%)")

    # Duplicados
    total_rows = df.count()
    unique_rows = df.distinct().count()
    duplicates = total_rows - unique_rows
    print(f"\nDUPLICADOS: {duplicates}")

    return df

spark_quality_analysis(df_spark)


 ANÁLISIS CALIDAD DATOS   
Numero de total registros: 541909
Total columnas: 8

VALORES NULOS 
Description: 1454 (0.27%)
CustomerID: 135080 (24.93%)

DUPLICADOS: 5268


DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: int, Country: string]

In [9]:
# Limpieza Distribuida Sapark

def spark_data_cleaning(df):
    """Pipeline de limpieza para Spark"""

    # 1. Eliminar registros con muchos nulls
    threshold = 0.5  # Eliminar filas con >50% nulls
    min_non_null = int(threshold * len(df.columns))
    df_clean = df.dropna(thresh=min_non_null)

    # 2. Imputación por columna
    numeric_cols = [field.name for field in df_clean.schema.fields
                   if field.dataType in [IntegerType(), DoubleType(), FloatType()]]

    # Imputar con mediana
    for col_name in numeric_cols:
        median_val = df_clean.approxQuantile(col_name, [0.5], 0.01)[0]
        df_clean = df_clean.na.fill({col_name: median_val})

    # Imputar categóricas con moda
    string_cols = [field.name for field in df_clean.schema.fields
                  if field.dataType == StringType()]


    for col_name in string_cols:
        print(col_name)
        mode_val = df_clean.filter(col(col_name).isNotNull()).groupBy(col_name).count().orderBy(desc("count")).first()[0]
        df_clean = df_clean.na.fill({col_name: mode_val})

    # 3. Eliminar duplicados
    df_clean = df_clean.dropDuplicates()

    # 4. Detección de outliers usando IQR
    for col_name in numeric_cols:
        quantiles = df_clean.approxQuantile(col_name, [0.25, 0.75], 0.01)
        q1, q3 = quantiles[0], quantiles[1]
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        df_clean = df_clean.filter(
            (col(col_name) >= lower_bound) &
            (col(col_name) <= upper_bound)
        )

    return df_clean


df_spark_clean = spark_data_cleaning(df_spark)
print(f"Registros después de limpieza: {df_spark_clean.count()}")
df_spark_clean.show(5)


InvoiceNo
StockCode
Description
InvoiceDate
Country
Registros después de limpieza: 468260
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536367|    22745|POPPY'S PLAYHOUSE...|       6| 12/1/2010 8:34|      2.1|     13047|United Kingdom|
|   536368|    22960|JAM MAKING SET WI...|       6| 12/1/2010 8:34|     4.25|     13047|United Kingdom|
|   536388|    22915|ASSORTED BOTTLE T...|      12| 12/1/2010 9:59|     0.42|     16250|United Kingdom|
|   536401|    21464|DISCO BALL ROTATO...|       1|12/1/2010 11:21|     4.25|     15862|United Kingdom|
|   536412|    22569|FELTCRAFT CUSHION...|       2|12/1/2010 11:49|     3.75|     17920|United Kingdom|
+---------+---------+--------------------+--------+---------------+---------+-